In [1]:
import json

import pandas as pd
from pathlib import Path

In [141]:
df = pd.read_json('maestro-v2.0.0.json')
# Sonata in F Major, K332 (1st movement)
# df[df["canonical_composer"]=="Johann Sebastian Bach"]
# df[df['canonical_composer'].str.contains("Johann Sebastian Bach")]

In [129]:
for f in df[df["canonical_composer"]=="Johann Sebastian Bach"].audio_filename:
    print(f)

2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_19_R1_2018_wav--1.wav
2014/MIDI-UNPROCESSED_04-05_R1_2014_MID--AUDIO_05_R1_2014_wav--1.wav
2015/MIDI-Unprocessed_R2_D2-19-21-22_mid--AUDIO-from_mp3_19_R2_2015_wav--1.wav
2009/MIDI-Unprocessed_06_R1_2009_03-07_ORIG_MID--AUDIO_06_R1_2009_06_R1_2009_03_WAV.wav
2009/MIDI-Unprocessed_06_R1_2009_03-07_ORIG_MID--AUDIO_06_R1_2009_06_R1_2009_04_WAV.wav
2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_08_R1_2015_wav--1.wav
2018/MIDI-Unprocessed_Recital5-7_MID--AUDIO_06_R1_2018_wav--1.wav
2013/ORIG-MIDI_01_7_10_13_Group_MID--AUDIO_08_R3_2013_wav--2.wav
2008/MIDI-Unprocessed_08_R1_2008_01-05_ORIG_MID--AUDIO_08_R1_2008_wav--1.wav
2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.wav
2004/MIDI-Unprocessed_SMF_22_R1_2004_01-04_ORIG_MID--AUDIO_22_R1_2004_03_Track03_wav.wav
2004/MIDI-Unprocessed_SMF_22_R1_2004_01-04_ORIG_MID--AUDIO_22_R1_2004_05_Track05_wav.wav
2014/MIDI-UNPROCESSED_16-18_R1_2014_MID--AUDIO_18_R1_2014_w

[{'audio_filename': '2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.wav',
  'canonical_composer': 'Alban Berg',
  'canonical_title': 'Sonata Op. 1',
  'duration': 698.661160312,
  'midi_filename': '2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi',
  'split': 'train',
  'year': 2018},
 {'audio_filename': '2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.wav',
  'canonical_composer': 'Alban Berg',
  'canonical_title': 'Sonata Op. 1',
  'duration': 759.51847125,
  'midi_filename': '2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi',
  'split': 'train',
  'year': 2008},
 {'audio_filename': '2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.wav',
  'canonical_composer': 'Alban Berg',
  'canonical_title': 'Sonata Op. 1',
  'duration': 464.649432608,
  'midi_filename': '2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi',
  'split': 'train',
  'ye

In [13]:
from pathlib import Path
import pretty_midi as pm

p1 = Path('C:/Users/fosca/Desktop/CNAM/datasets/maestro-v2.0.0/2004')
p2 = Path('C:/Users/fosca/Desktop/CNAM/datasets/maestro-v2.0.0/performances_2004')
print(len([x for x in p1.iterdir()]))
print(len([x for x in p2.iterdir()]))

132
343


In [27]:
file1 = Path(p1,"MIDI-Unprocessed_XP_21_R1_2004_03_ORIG_MID--AUDIO_21_R1_2004_04_Track04_wav.midi")
midi = pm.PrettyMIDI(str(file1))
midi.get_end_time().time_to_tick()

AttributeError: 'numpy.float64' object has no attribute 'time_to_tick'

## Crawling

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import difflib

In [81]:
def web_page_to_mapping(url,year):
    page = urlopen(url)
    html = page.read()
    soup = BeautifulSoup(html, 'html.parser')
    #find the big sections for each artist
    tables = soup.findAll("table" , {"class": "detail-text"})
    mapping = []
    for t in tables:
        tds = t.findAll("td")
        for i,td in enumerate(tds[:-1]):
            next_children = tds[i+1].findChildren("a" , recursive=False)
            if len(next_children) ==1:
                # element is (performer,author,year,title)
                mapping.append((next_children[0]["href"].split("/")[-1],td.text,year,re.sub('\s+', ' ', next_children[0].text).strip()))
    return mapping

In [108]:
urls = ["http://www.piano-e-competition.com/midi_2002.asp",
        "http://www.piano-e-competition.com/midi_2004.asp",
        "http://www.piano-e-competition.com/midi_2006.asp",
        "http://www.piano-e-competition.com/midi_2008.asp",
        "http://www.piano-e-competition.com/midi_2009.asp",
        "http://www.piano-e-competition.com/midi_2011.asp",
        "http://www.piano-e-competition.com/midi_2013.asp",
        "http://www.piano-e-competition.com/midi_2014.asp",
        "http://www.piano-e-competition.com/midi_2015.asp",
        "http://www.piano-e-competition.com/midi_2017.asp",
        "http://www.piano-e-competition.com/midi_2018.asp"]

years = [2002,2004,2006,2008,2009,2011,2013,2014,2015,2017,2018]

# mapping = []
for i_url,url in enumerate(urls):
    to_add = web_page_to_mapping(url,years[i_url])
    mapping.extend(to_add)
    
#mapping is a 4-tuple: yamaha_midi_name, author, year, title
#the idea is that the title used in google will be the same that is used in yamaha

In [84]:
mapping

[('chan01.mid', 'Schubert', 2002, 'Sonata in G Major, Op. 87, D894'),
 ('chan02.mid', 'Ravel', 2002, 'Alborada del Gracioso'),
 ('cho01.mid', 'Busoni', 2002, "Sonatina 'Carmen'"),
 ('cho02.mid', 'Chopin', 2002, 'Fantasy in F Minor, Op. 49'),
 ('cho03.mid', 'Schumann', 2002, 'Symphonic Etudes, Op. 13'),
 ('cho04.mid', 'Debussy', 2002, 'Images II'),
 ('cho05.mid', 'Balakirev', 2002, 'Islamei'),
 ('dossin01.mid', 'Mozart', 2002, 'Variations in D Major, K573'),
 ('dvorkine01.mid', 'Schumann-Liszt', 2002, "'Widmung'"),
 ('dvorkine02.mid', 'Chopin', 2002, "'Polonaise Fantasy', Op. 61"),
 ('dvorkine03.mid', 'Liszt', 2002, 'Sonata in B Minor'),
 ('dvorkine04.mid', 'Prokofiev', 2002, 'Sonata No. 7, Op. 83'),
 ('franceschetti01.mid', 'Schumann', 2002, 'Davidsbundlertanze, Op. 6 (1837)'),
 ('franceschetti02.mid', 'Ravel', 2002, 'La Valse'),
 ('franceschetti03.mid', 'Mussorgsky', 2002, 'Pictures at an Exhibition'),
 ('giltburg01.mid', 'Bach-Busoni', 2002, 'Chaconne in D Minor'),
 ('giltburg02.mid'

# Crawling the bigger website

In [95]:
# http://www.yamahaden.com/midi-files/advanced-search/9431

def web_page2_to_mapping(url):
    page = urlopen(url)
    html = page.read()
    soup = BeautifulSoup(html, 'html.parser')
    #find the big sections for each artist
    piece_div = soup.findAll("div" , {"class": "teaser-item"})
    mapping = []
    for d in piece_div:
        try:
            #find the title and the author
            head_section = d.find("div" , {"class": "piece_fields"})
            title_parts = list(head_section.findAll("span"))
            if len(title_parts) == 0:
                title = None
                author = None
            else:
                title = ''.join([t.text for t in title_parts if ((t.text!="  By ") and (t["class"][0]!="composer_name")) ])
                # ?? should we consider if t["class"] == 'item_subcont_n' ??
                author = [t.text for t in title_parts if (t["class"][0]=="composer_name")][0]
            #find the performer name
            performer = [e.find("a").text for e in d.findAll("li" , {"class": "element element-download"}) if e.find("strong").text == "Standard MIDI File: "][0]
            #find the year
            year = [e.text.split()[-1] for e in d.findAll("li" , {"class": "element element-radio"}) if e.find("strong").text == "Recording Date: "][0]

            mapping.append((performer,author,year,title))
        except:
            print("Failed for page {} and div {}".format(url, d))
    return mapping

In [80]:
# page = urlopen("http://www.yamahaden.com/midi-files/advanced-search/9431")
# html = page.read()
# soup = BeautifulSoup(html, 'html.parser')
# #find the big sections for each artist
# piece_div = soup.findAll("div" , {"class": "teaser-item"})
# mapping = []
# # for d in piece_div:
# d = piece_div[6]
# #find the title and the author
# head_section = d.find("div" , {"class": "piece_fields"})
# title_parts = list(head_section.findAll("span"))
# if len(title_parts) == 0:
#     title = None
#     author = None
# else:
#     title = ''.join([t.text for t in title_parts if ((t.text!="  By ") and (t["class"][0]!="composer_name")) ])
#     # ?? should we consider if t["class"] == 'item_subcont_n' ??
#     author = [t.text for t in title_parts if (t["class"][0]=="composer_name")][0]
# #find the performer name
# performer = [e.find("a").text for e in d.findAll("li" , {"class": "element element-download"}) if e.find("strong").text == "Standard MIDI File: "][0]
# #find the year
# year = [e.text.split()[-1] for e in d.findAll("li" , {"class": "element element-radio"}) if e.find("strong").text == "Recording Date: "][0]


In [90]:
m = web_page2_to_mapping("http://www.yamahaden.com/midi-files/advanced-search/9431")

In [96]:
urls =["http://www.yamahaden.com/midi-files/advanced-search/9431"]
for page_number in range(2,188+1):
    urls.append("http://www.yamahaden.com/midi-files/advanced-search/9431?page={}".format(page_number))

mapping = []
for url in urls:
    mapping.extend(web_page2_to_mapping(url))
    
#it fails for some pieces, but those have n

Failed for page http://www.yamahaden.com/midi-files/advanced-search/9431?page=18 and div <div class="teaser-item">
<div class="piece_fields"> <span class="item_subcont">Piano Sonata No. 04 in E-Flat Major</span> <span class="item_subcont item_subcont_n">, Op. 7</span> <br/><span class="item_subcont">“Grand Sonata”</span><br/><span class="item_subcont">  By </span><span class="composer_name"><a href="/midi-files/category/ludwig_van_beethoven">Ludwig Beethoven</a></span></div>
<div class="piece_fields"><span>Date Of Composition: </span><span>1797</span></div><div class="perform_block">
<ul class="pos-specification">
</ul>
</div>
</div>
Failed for page http://www.yamahaden.com/midi-files/advanced-search/9431?page=28 and div <div class="teaser-item">
<div class="piece_fields"> <span class="item_subcont">Piano Sonata No. 18 in E-Flat Major</span> <span class="item_subcont item_subcont_n">, Op. 31, No. 3</span><br/><span class="item_subcont">  By </span><span class="composer_name"><a href="/

In [125]:
print(len(mapping))


cleaned_mapping = [m for m in mapping if m[1]!=None]
print(len(cleaned_mapping))

4943
3765


#### Let's check if all google pieces have a yamaha title that we extract

In [126]:
mag_df = pd.read_json('maestro-v2.0.0.json')
mag_df
matched = []
unmatched = []

num2string = {"0":"zero", "1":"one", "2":"two", "3":"three", "4":"four", "5":"five", "6":"six",
                "7":"seven", "8":"eight", "9":"nine"}
def clean_string(string): #function to lowercase, delete spaces and transform numbers to letters
    #numbers to string
    for e in num2string:
        string = string.replace(e, num2string[e])
    # delete spaces
    string = string.replace(" ","")
    #lowercase
    string = string.lower()
    #delete commas, points
    string = string.replace(",","").replace(".","")
    return string

for i,row in mag_df.iterrows():
    # we ignore the spaces in the matching
    possible_matches = [m for m in cleaned_mapping if clean_string(m[3])==clean_string(row["canonical_title"])]
    if len(possible_matches) == 1:
        matched.append(row["canonical_title"])
    elif len(possible_matches) == 0:
        unmatched.append(row["canonical_title"])
    else:
        matched.append(row["canonical_title"])

In [127]:
print("number:", len(unmatched))
print(unmatched)

number: 290
['Etude Op. 2 No.1;  Etudes Op. 8, Nos. 5, 11 and 12', 'Etude Op. 42, Nos. 4 & 5', 'Etude Op. 8, No. 13', 'Etudes from Op.8', 'Fantasy Op. 28 in B Minor', 'Sonata No. 2', 'Sonata No. 3 in F-sharp Minor, Op. 23', 'Sonata No. 3 in F-sharp Minor, Op. 23', 'Sonata No. 3 in F-sharp Minor, Op. 23', 'Sonata No. 9', 'Sonata No. 9', 'Piano Trio in D Minor, Op. 32', 'Piano Trio in D Minor, Op. 32', 'Piano Trio in D Minor, Op. 32', 'Faust Waltz', '"Images", Series I', '"Les collines d\'Anacapri" from Preludes, Book I', '"Voiles" from Preludes, Book I', 'Bruyeres', 'Etude "Pour le arpeges composes"', 'Etude "Pour les accords"', 'Etude "Pour les huit doigts"', 'Etude No. 7 "Study in Chromatic Steps"', "Feux d'artifice", 'Preludes No. 2, 6, 12', 'Preludes No. 2, 6, 12', 'Preludes, Book II, III - La puerta del vino', 'Preludes, Book II, VII - La terasse des audiences du clair de lune', 'PreludesBII/7-12', 'Three Etudes', 'Prel./Chor./Fug.', 'Sonata in B Minor, K. 27', 'Sonata, K239', 'Pia

In [20]:
# mag_df[mag_df['canonical_title'].str.contains("Sonata in F")]
difflib.get_close_matches('Nocturne in Db', [m[3] for m in mapping])

['Nocturne Op. 33']

### let's check if all our midi files are in yamaha crawled list

In [147]:
perf_df = pd.read_pickle(Path("C:/Users/fosca/Desktop/CNAM/performed-midi-dataset/performance_dataframe.pkl"))
# perf_df[perf_df["author"] == "Bach"].drop_duplicates(subset=["folder"])

In [110]:
matched = []
unmatched = []

for i,row in perf_df.iterrows():
    possible_matches = [m[0] for m  in mapping if m[0].lower()==(row.performer+".mid").lower()]  
    if len(possible_matches) == 1:
        matched.append(row["performed_midi_path"])
    elif len(possible_matches) == 0:
        unmatched.append(row["performed_midi_path"])
    else:
        matched.append(row["performed_midi_path"])

In [111]:
print("number matched:", len(matched))
print("number unmatched:", len(unmatched))
print(unmatched)

number matched: 1060
number unmatched: 44
['Bach/Fugue/bwv_883/Herbert02.mid', 'Bach/Prelude/bwv_883/Herbert02.mid', 'Beethoven/Piano_Sonatas/32-1/FALIKS01_2004.mid', 'Chopin/Ballades/1/KimSuyeon04.mid', 'Chopin/Ballades/1/LeeKoeun03.mid', 'Chopin/Ballades/1/LeeKoeun06.mid', 'Chopin/Ballades/1/LinTao04.mid', 'Chopin/Ballades/1/YeSijing01.mid', 'Chopin/Ballades/3/LinPeng06.mid', 'Chopin/Ballades/4/LuEric04.mid', 'Chopin/Etudes_op_10/1/LeeHanchien02.mid', 'Chopin/Etudes_op_10/1/Tysman02 (2).mid', 'Chopin/Etudes_op_10/10/Na_2009_02.mid', 'Chopin/Etudes_op_10/12/KimBenjamin04.mid', 'Chopin/Etudes_op_10/12/Na_2006_02.mid', 'Chopin/Etudes_op_10/12/ZhangYunling02.mid', 'Chopin/Etudes_op_10/2/JiaXin01.mid', 'Chopin/Etudes_op_10/2/YeSijing03.mid', 'Chopin/Etudes_op_10/3/SunMeiting08.mid', 'Chopin/Etudes_op_10/4/JiaRan_2009_02.mid', 'Chopin/Etudes_op_10/4/Richardson_2013_03.mid', 'Chopin/Etudes_op_10/5/Feiner_2006_02.mid', 'Chopin/Etudes_op_10/5/KimEunhae04.mid', 'Chopin/Etudes_op_10/8/ChenJie03

In [117]:
difflib.get_close_matches('Herbert02', [m[0] for m in mapping])

['Hebert02.MID', 'Hebert02.MID', 'Hebert04.MID']

In [120]:
[m for m in mapping if m[0]=="Hebert02.MID"]

[('Hebert02.MID',
  'Johann Sebastian Bach',
  '2013',
  'Prelude & Fugue No. 14 in F-Sharp Minor, BWV 883'),
 ('Hebert02.MID',
  'Johann Sebastian Bach',
  2013,
  'Prelude & Fugue No. 14 in F-Sharp Minor, WTC II')]

In [92]:
# check if our midifiles are unique
perf_df[perf_df.duplicated(['performer'],keep=False)]

,author,folder,midi2midi_alignment_path,performed_midi_path,performer,score2midi_alignment,score_midi_path,score_xml_path,title
0,Bach,Bach/Fugue/bwv_846,Bach/Fugue/bwv_846/Shi05_infer_corresp.txt,Bach/Fugue/bwv_846/Shi05.mid,Shi05,Bach/Fugue/bwv_846/Shi05_infer_match.txt,Bach/Fugue/bwv_846/midi_cleaned.mid,Bach/Fugue/bwv_846/musicxml_cleaned.musicxml,Fugue_bwv_846
1,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Denisova06_infer_corresp.txt,Bach/Fugue/bwv_848/Denisova06.mid,Denisova06,Bach/Fugue/bwv_848/Denisova06_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
2,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lee01_infer_corresp.txt,Bach/Fugue/bwv_848/Lee01.mid,Lee01,Bach/Fugue/bwv_848/Lee01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
3,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/LeeSH01_infer_corresp.txt,Bach/Fugue/bwv_848/LeeSH01.mid,LeeSH01,Bach/Fugue/bwv_848/LeeSH01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
4,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lin04_infer_corresp.txt,Bach/Fugue/bwv_848/Lin04.mid,Lin04,Bach/Fugue/bwv_848/Lin04_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
5,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lou01_infer_corresp.txt,Bach/Fugue/bwv_848/Lou01.mid,Lou01,Bach/Fugue/bwv_848/Lou01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
6,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/MiyashitaM01_infer_corresp.txt,Bach/Fugue/bwv_848/MiyashitaM01.mid,MiyashitaM01,Bach/Fugue/bwv_848/MiyashitaM01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
7,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Mizumoto03_infer_corresp.txt,Bach/Fugue/bwv_848/Mizumoto03.mid,Mizumoto03,Bach/Fugue/bwv_848/Mizumoto03_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
8,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/SunY01_infer_corresp.txt,Bach/Fugue/bwv_848/SunY01.mid,SunY01,Bach/Fugue/bwv_848/SunY01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
9,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Zhou01_infer_corresp.txt,Bach/Fugue/bwv_848/Zhou01.mid,Zhou01,Bach/Fugue/bwv_848/Zhou01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848


#### now that we have our alignment, let's link our performances to magenta pieces

In [121]:
# import our performance dataset
perf_df = pd.read_pickle(Path("C:/Users/fosca/Desktop/CNAM/performed-midi-dataset/performance_dataframe.pkl"))
# import magenta df
mag_df = pd.read_json('maestro-v2.0.0.json')

In [122]:
perf_df.head()

,author,folder,midi2midi_alignment_path,performed_midi_path,performer,score2midi_alignment,score_midi_path,score_xml_path,title
0,Bach,Bach/Fugue/bwv_846,Bach/Fugue/bwv_846/Shi05_infer_corresp.txt,Bach/Fugue/bwv_846/Shi05.mid,Shi05,Bach/Fugue/bwv_846/Shi05_infer_match.txt,Bach/Fugue/bwv_846/midi_cleaned.mid,Bach/Fugue/bwv_846/musicxml_cleaned.musicxml,Fugue_bwv_846
1,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Denisova06_infer_corresp.txt,Bach/Fugue/bwv_848/Denisova06.mid,Denisova06,Bach/Fugue/bwv_848/Denisova06_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
2,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lee01_infer_corresp.txt,Bach/Fugue/bwv_848/Lee01.mid,Lee01,Bach/Fugue/bwv_848/Lee01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
3,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/LeeSH01_infer_corresp.txt,Bach/Fugue/bwv_848/LeeSH01.mid,LeeSH01,Bach/Fugue/bwv_848/LeeSH01_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848
4,Bach,Bach/Fugue/bwv_848,Bach/Fugue/bwv_848/Lin04_infer_corresp.txt,Bach/Fugue/bwv_848/Lin04.mid,Lin04,Bach/Fugue/bwv_848/Lin04_infer_match.txt,Bach/Fugue/bwv_848/midi_cleaned.mid,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Fugue_bwv_848


In [154]:
mag_df.head()

,audio_filename,canonical_composer,canonical_title,duration,midi_filename,split,year
0,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,Alban Berg,Sonata Op. 1,698.661160,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,train,2018
1,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,Alban Berg,Sonata Op. 1,759.518471,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,train,2008
2,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,Alban Berg,Sonata Op. 1,464.649433,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,train,2017
3,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",872.640588,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,train,2004
4,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,Alexander Scriabin,"3 Etudes, Op. 65",397.857508,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,validation,2006


In [167]:
def our2mag_map(our_df,mag_df,t2p_map):
    for our_i,our_row in our_df.iterrows():
        possible_title = [m[3] for m  in t2p_map if m[0]==our_row.performer+".mid"]
        print("possible_title: ", possible_title)
        if len(possible_title) == 1:
            corresp = list(mag_df[mag_df["canonical_title"]==possible_title[0]]["midi_filename"])
            print("corresp:", corresp)
            if len(corresp) == 1:
                print(our_row.performed_midi_path, "-->", corresp[0])
            elif len(corresp) == 0:
                print("no corresp")
            else:
                print("multiple corresp")
        else:
            print("multiple possible titles: ")

In [168]:
our2mag_map(perf_df,mag_df,mapping)

possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_titl

possible_title:  ['Islamei']
corresp: ['2004/MIDI-Unprocessed_XP_04_R1_2004_06_ORIG_MID--AUDIO_04_R1_2004_08_Track08_wav.midi']
Balakirev/Islamey/CHEN04.mid --> 2004/MIDI-Unprocessed_XP_04_R1_2004_06_ORIG_MID--AUDIO_04_R1_2004_08_Track08_wav.midi
possible_title:  ['Islamei']
corresp: ['2004/MIDI-Unprocessed_XP_04_R1_2004_06_ORIG_MID--AUDIO_04_R1_2004_08_Track08_wav.midi']
Balakirev/Islamey/CHEN10.mid --> 2004/MIDI-Unprocessed_XP_04_R1_2004_06_ORIG_MID--AUDIO_04_R1_2004_08_Track08_wav.midi
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Islamey']
corresp: ['2006/MIDI-Unprocessed_13_R1_2006_01-06_ORIG_MID--AUDIO_13_R1_2006_05_Track05_wav.midi']
Balakirev/Islamey/Na04.mid --> 2006/MIDI-Unprocessed_13_R1_2006_01-06_ORIG_MID--AUDIO_13_R1_2006_05_Track05_wav.midi
possible_title:  []
multiple possible titles: 
possible_title:  ['Islamey']
corresp: ['2006/MIDI-Unprocessed_13_R1_2006_01-06_ORIG_MID--AUDIO_13_R1_2006_05_Track05_wav

possible_title:  ['Sonata in F minor Op. 57 "Appassionata", 1st mvmt']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in F Minor, Opus 57 (1st Mov.)']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in E Minor, Hob. XVI:34, 1st mvmt']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in F Minor, Op. 57 "Appassionata", 1st mvmt']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple poss

possible_title:  ['Sonata in A Major, K331, 3rd mvmt']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in C Minor, Opus 111, 1st mvmt']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in C Minor, Opus 111 (1st Mov.)']
corresp: []
no corresp
possible_title:  ['Sonata in C Minor, Op. 111, 1st mvmt']
corresp: []
no corresp
possible_title:  ['Sonata in C Minor, Opus 111 (1st Mov.)']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in E flat Major, Op. 7 (1st Mov.)']
corresp: []
no corresp
possible_title:  ['Sonata in E-flat Major, Op. 7, 1st mvmt']
corresp: []
no corresp
p

possible_title:  ['Etude Op. 10 No. 1']
corresp: ['2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--2.midi', '2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_10_R1_2015_wav--2.midi', '2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--4.midi', '2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_20_R1_2015_wav--3.midi', '2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_19_R1_2013_wav--3.midi']
multiple corresp
possible_title:  ['Etude Op. 10 No. 1']
corresp: ['2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--2.midi', '2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_10_R1_2015_wav--2.midi', '2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_12_R1_2015_wav--4.midi', '2015/MIDI-Unprocessed_R1_D2-13-20_mid--AUDIO-from_mp3_20_R1_2015_wav--3.midi', '2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_19_R1_2013_wav--3.midi']
multiple corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
po

possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Etude Opus 10, No. 8']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Etude No. 10, Opus 25']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Etude Op. 25, No. 10']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in A Major, Op. 101, 3rd m

possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata No. 3 in B Minor, Op. 58']
corresp: ['2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.midi', '2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_07_Track07_wav.midi', '2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_08_Track08_wav.midi', '2018/MIDI-Unprocessed_Recital13-15_MID--AUDIO_13_R1_2018_wav--3.midi', '2018/MIDI-Unprocessed_Chamber5_MID--AUDIO_18_R3_2018_wav--1.midi']
multiple corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata in B Minor, Op. 58']
corresp: ['2004/MIDI-Unprocessed_XP_18_R1_2004_04_ORIG_MID--AUDIO_18_R1_2004_07_Track07_wav.midi']
Chopin/Sonata_3/4th/SCHU09.mid --> 2004/MIDI-Unprocessed_XP_18_R1_2004_04_ORIG_MID--A

possible_title:  ['Mephisto Waltz No. 1']
corresp: ['2004/MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi', '2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_10_Track10_wav.midi', '2009/MIDI-Unprocessed_17_R1_2009_01-03_ORIG_MID--AUDIO_17_R1_2009_17_R1_2009_03_WAV.midi', '2013/ORIG-MIDI_02_7_8_13_Group__MID--AUDIO_11_R2_2013_wav--3.midi']
multiple corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Mephisto Waltz, No. 1']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['Mephisto Waltz No. 1']
corresp: ['2004/MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi', '2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORIG_MID--AUDIO_12_R1_2004_10_Track10_wav.midi', '2009/MIDI-Unprocessed_17_R1_2009_01-03_ORIG_

possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['"Ondine" from Gaspard de la nuit']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  ['"Ondine" from Gaspard de la nuit']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 


possible_title:  []
multiple possible titles: 
possible_title:  ['Barcarolle in F-sharp Major, Op. 60']
corresp: ['2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_18_R3_2013_wav--1.midi']
Scriabin/Sonatas/5/Na03.MID --> 2013/ORIG-MIDI_03_7_10_13_Group_MID--AUDIO_18_R3_2013_wav--1.midi
possible_title:  []
multiple possible titles: 
possible_title:  ['Sonata No. 5, Opus 53']
corresp: []
no corresp
possible_title:  []
multiple possible titles: 
possible_title:  []
multiple possible titles: 


# Match the bach pieces with our performances
In particular we try to match the preludes, then we need to split the fugues from them

In [28]:
maestro_df = pd.read_excel("bach_perf_maestro.xlsx")
maestro_df

,author,folder,score_midi_path,score_xml_path,Unnamed: 4
0,Bach,Bach/Italian_concerto,Bach/Italian_concerto/midi_cleaned.mid,Bach/Italian_concerto/musicxml_cleaned.musicxml,None
1,Bach,Bach/Prelude/bwv_846,Bach/Prelude/bwv_846/midi_cleaned.mid,Bach/Prelude/bwv_846/musicxml_cleaned.musicxml,2014/MIDI-UNPROCESSED_16-18_R1_2014_MID--AUDIO...
2,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2011/MIDI-Unprocessed_24_R1_2011_MID--AUDIO_R1...
3,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2011/MIDI-Unprocessed_04_R1_2011_MID--AUDIO_R1...
4,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2011/MIDI-Unprocessed_12_R1_2011_MID--AUDIO_R1...
5,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2013/ORIG-MIDI_01_7_7_13_Group__MID--AUDIO_13_...
6,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2008/MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MI...
7,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2008/MIDI-Unprocessed_10_R1_2008_01-04_ORIG_MI...
8,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-fr...
9,Bach,Bach/Prelude/bwv_848,Bach/Prelude/bwv_848/midi_cleaned.mid,Bach/Prelude/bwv_848/musicxml_cleaned.musicxml,2013/ORIG-MIDI_01_7_7_13_Group__MID--AUDIO_12_...


In [19]:
virtuoso_df = pd.read_excel("performance_bach.xlsx")
virtuoso_df[[p["folder"].split("/")[1] == "Prelude" for i,p in virtuoso_df.iterrows()]]

,Unnamed: 0,author,folder,performer,done
88,88,Bach,Bach/Prelude/bwv_846,Shi05,True
89,89,Bach,Bach/Prelude/bwv_848,Denisova06,True
90,90,Bach,Bach/Prelude/bwv_848,Lee01,True
91,91,Bach,Bach/Prelude/bwv_848,LeeSH01,True
92,92,Bach,Bach/Prelude/bwv_848,Lin04,True
93,93,Bach,Bach/Prelude/bwv_848,Lou01,True
94,94,Bach,Bach/Prelude/bwv_848,MiyashitaM01,True
95,95,Bach,Bach/Prelude/bwv_848,Mizumoto03,True
96,96,Bach,Bach/Prelude/bwv_848,SunY01,True
97,97,Bach,Bach/Prelude/bwv_848,Zhou01,True


In [39]:
#create a dictionary with all the midi performances from virtuosonet
virtuoso_dict = {}
for i,row in virtuoso_df.iterrows():
    key = row["folder"].split("/")[-1]
    if key not in virtuoso_dict:
        virtuoso_dict[key] = []
    virtuoso_dict[key].append(row["folder"] +"/"+row["performer"]+".mid")    
        
#create a dictionary with all the midi performances from maestro
maestro_dict = {}
for i,row in maestro_df.iterrows():
    key = row["folder"].split("/")[-1]
    if key not in maestro_dict:
        maestro_dict[key] = []
    maestro_dict[key].append(row["Unnamed: 4"])
    
maestro_dict = {key:value for key,value in maestro_dict.items() if value!=["None"]}

In [40]:
maestro_dict

{'bwv_846': ['2014/MIDI-UNPROCESSED_16-18_R1_2014_MID--AUDIO_16_R1_2014_wav--1.midi'],
 'bwv_848': ['2011/MIDI-Unprocessed_24_R1_2011_MID--AUDIO_R1-D9_08_Track08_wav.midi',
  '2011/MIDI-Unprocessed_04_R1_2011_MID--AUDIO_R1-D2_02_Track02_wav.midi',
  '2011/MIDI-Unprocessed_12_R1_2011_MID--AUDIO_R1-D4_15_Track15_wav.midi',
  '2013/ORIG-MIDI_01_7_7_13_Group__MID--AUDIO_13_R1_2013_wav--1.midi',
  '2008/MIDI-Unprocessed_02_R1_2008_01-05_ORIG_MID--AUDIO_02_R1_2008_wav--1.midi',
  '2008/MIDI-Unprocessed_10_R1_2008_01-04_ORIG_MID--AUDIO_10_R1_2008_wav--1.midi',
  '2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_09_R1_2015_wav--1.midi',
  '2013/ORIG-MIDI_01_7_7_13_Group__MID--AUDIO_12_R1_2013_wav--1.midi',
  '2017/MIDI-Unprocessed_059_PIANO059_MID--AUDIO-split_07-07-17_Piano-e_2-03_wav--1.midi',
  '2011/MIDI-Unprocessed_08_R1_2011_MID--AUDIO_R1-D3_07_Track07_wav.midi',
  '2017/MIDI-Unprocessed_049_PIANO049_MID--AUDIO-split_07-06-17_Piano-e_2-06_wav--1.midi'],
 'bwv_854': ['2013/ORIG-MIDI_0